# Indian Cricket Team ODI analysis

This is the implementation of the "INCREASED PREDICTION ACCURACY IN THE GAME OF CRICKET USING MACHINE LEARNING" research paper. The main agenda of the paper is to take the career Data of each player and predict the number of wickets they will take or the number of runs they will make, given the circumstances of the next matches.

The Paper uses the data of all the players in cricet from 2005 to 2017. As i dont have the recources to store that much data. I have taken the data of the 27 current Indian players from the start of their career till 2020. The most senior player we have is Mahendra Singh Dhoni whose data is from 2002-2020.

The Players are Divided into 3 Categories-
1. Batsmen
2. Bowlers
3. All Rounders

In [49]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import statistics 
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics 
from sklearn.utils import shuffle
%matplotlib inline
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import warnings
import os
warnings.filterwarnings('ignore')


In [2]:
# Overall Stats for all batsmen
column_name = ['Name','playing_span', 'matches_played', 'innings_batted', 'not_outs','runs_scored','highest_innings_score',
               'batting_average','balls_faced','batting_strike_rate','hundreds_scored', 'fifties','ducks','fours','sixes']
df_overall_bat = pd.DataFrame(columns=column_name)
df_overall_bat

,Name,playing_span,matches_played,innings_batted,not_outs,runs_scored,highest_innings_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties,ducks,fours,sixes


In [3]:
# Innings by innings stats for each batsmen
column_name =['Runs','Mins','BF','4s','6s','SR','Pos','Dismissal','Inns','Opposition','Ground']
df_personal_bat= pd.DataFrame(columns=column_name)
df_personal_bat

,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground


In [4]:
# Overall Bowling stats for all Bowlers 
column_name=['Overs','Mdns','Runs','Wkts','Econ','Pos','Inns','Opposition','Ground']
df_personal_ball= pd.DataFrame(columns=column_name)
df_personal_ball

,Overs,Mdns,Runs,Wkts,Econ,Pos,Inns,Opposition,Ground


In [5]:
# Inning by Inning Stats By each Bowler
column_name=['Name','Span','Mat','Inns','Overs','Mdns','Runs','Wkts','best_bowling_in_innings','Ave','Econ','SR','4s','5s']
df_overall_ball= pd.DataFrame(columns=column_name)
df_overall_ball

,Name,Span,Mat,Inns,Overs,Mdns,Runs,Wkts,best_bowling_in_innings,Ave,Econ,SR,4s,5s


## Function for Extracting the overall and the Inning by Inning stats of each player.

This function takes URL and two dataframes one of the overall stats and the other of the innings by innings stats of each of the batsman and the bowler.

The function outputs 1 CSV of the players inning by inning stats and 1 dataframe of the players and their overall stats.

In [6]:
def extract_data(Name,url1,url2, df1, df2,x):
    user_agent = {"User-Agent": "Mozilla/5.0 "
                            "(Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/80.0.3987.163 Safari/537.36"}
    if (x==0):# If the player is a Batsman
        source = requests.get(url1, headers = user_agent).text
        soup = BeautifulSoup(source,"lxml")

        items = soup.find("div", id="ciMainContainer")
        Bat_Overall= items.find_all("table",{"class":"engineTable"})[2]
        Bat_personal= items.find_all("table",{"class":"engineTable"})[3]
        rows= Bat_Overall.find_all("td")
        df1= df1.append({'Name':Name,'playing_span':rows[1].text, 'matches_played':rows[2].text, 'innings_batted':rows[3].text, 'not_outs':rows[4].text,
                        'runs_scored':rows[5].text,'highest_innings_score':rows[6].text, 'batting_average':rows[7].text,'balls_faced':rows[8].text,
                        'batting_strike_rate':rows[9].text,'hundreds_scored':rows[10].text, 'fifties':rows[11].text,'ducks':rows[12].text,'fours':rows[13].text,
                        'sixes':rows[14].text}, ignore_index= True)
        row= Bat_personal.find_all("td")
        for a in range(0, len(row)-1, 14):
            df2= df2.append({'Runs':row[a].text,'Mins':row[a+1].text,'BF':row[a+2].text,'4s':row[a+3].text,'6s':row[a+4].text,'SR':row[a+5].text,
                             'Pos':row[a+6].text,'Dismissal':row[a+7].text,'Inns':row[a+8].text,'Opposition':row[a+10].text,'Ground':row[a+11].text},ignore_index=True)
        df2.to_csv(Name+'.csv')
        return df1,x
    
    if(x==1): #If the Player is a Bowler
        source = requests.get(url2, headers = user_agent).text
        soup = BeautifulSoup(source,"lxml")

        items = soup.find("div", id="ciMainContainer")
        Ball_Overall= items.find_all("table",{"class":"engineTable"})[2]
        Ball_personal= items.find_all("table",{"class":"engineTable"})[3]
        rows= Ball_personal.find_all("td")
        for i in range(0, len(rows)-1, 12):
            df2= df2.append({'Overs':rows[i].text,'Mdns':rows[i+1].text,'Runs':rows[i+2].text,'Wkts':rows[i+3].text,'Econ':rows[i+4].text,
                             'Pos':rows[i+5].text,'Inns':rows[i+6].text,'Opposition':rows[i+8].text,'Ground':rows[i+9].text},ignore_index=True)
        row= Ball_Overall.find_all("td")
        df1= df1.append({'Name':Name,'Span':row[1].text,'Mat':row[2].text,'Inns':row[3].text,'Overs':row[4].text,'Mdns':row[5].text,'Runs':row[6].text,'Wkts':row[7].text,
                         'best_bowling_in_innings':row[8].text,'Ave':row[9].text,'Econ':row[10].text,'SR':row[11].text,'4s':row[12].text,'5s':row[13].text}, ignore_index= True)
        
        df2.to_csv(Name+'.csv')
        return df1,x
    
    

#### The Bowling areas of the Batsmen and the Batting areas of the bowlers is not taken into account. This is so because the dataset would largely be filled with null values. The all rounders are passed through the functions twice because their both batting and the bowling data is taken into account. 

In [9]:
all_players=[["Virat Kohli","https://stats.espncricinfo.com/ci/engine/player/253802.html?class=2;template=results;type=batting;view=innings", "",0,df_overall_bat,df_personal_bat],
            ["Rohit Sharma","https://stats.espncricinfo.com/ci/engine/player/34102.html?class=2;template=results;type=batting;view=innings", "",0,df_overall_bat,df_personal_bat],
            ["Ajinkya Rahane","https://stats.espncricinfo.com/ci/engine/player/277916.html?class=2;template=results;type=batting;view=innings", "",0,df_overall_bat,df_personal_bat],
            ['Mayank Agarwal','https://stats.espncricinfo.com/ci/engine/player/398438.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Prithvi Shaw','https://stats.espncricinfo.com/ci/engine/player/398438.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Shikhar Dhawan','https://stats.espncricinfo.com/ci/engine/player/28235.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['KL Rahul', 'https://stats.espncricinfo.com/ci/engine/player/422108.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Shreyas Iyer','https://stats.espncricinfo.com/ci/engine/player/642519.html?class=3;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Manish Pandey','https://stats.espncricinfo.com/ci/engine/player/290630.html?class=3;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Cheteshwar Pujara','https://stats.espncricinfo.com/ci/engine/player/32540.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['MS Dhoni','https://stats.espncricinfo.com/ci/engine/player/28081.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Dinesh Karthik','https://stats.espncricinfo.com/ci/engine/player/30045.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Rishabh Pant','https://stats.espncricinfo.com/ci/engine/player/931581.html?class=2;template=results;type=batting;view=innings', "",0,df_overall_bat,df_personal_bat],
            ['Deepak Chahar','','https://stats.espncricinfo.com/ci/engine/player/447261.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Bhuvaneshwar Kumar',"",'https://stats.espncricinfo.com/ci/engine/player/326016.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Shradul Thakur','','https://stats.espncricinfo.com/ci/engine/player/475281.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Navdeep Saini','','https://stats.espncricinfo.com/ci/engine/player/700167.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Mohammed Shami','','https://stats.espncricinfo.com/ci/engine/player/481896.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Ishant Sharma','','https://stats.espncricinfo.com/ci/engine/player/236779.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Umesh Yadav','','https://stats.espncricinfo.com/ci/engine/player/376116.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Yuzvendra Chahal','','https://stats.espncricinfo.com/ci/engine/player/430246.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Kuldeep Yadav','','https://stats.espncricinfo.com/ci/engine/player/559235.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Jasprit Bumrah','','https://stats.espncricinfo.com/ci/engine/player/625383.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['R. Ashwin bat','https://stats.espncricinfo.com/ci/engine/player/26421.html?class=2;template=results;type=batting;view=innings',"",0,df_overall_bat,df_personal_bat],
            ['R. Ashwin bowl','','https://stats.espncricinfo.com/ci/engine/player/26421.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Ravindra Jadeja bat','https://stats.espncricinfo.com/ci/engine/player/234675.html?class=2;template=results;type=batting;view=innings',"",0,df_overall_bat,df_personal_bat],
            ['Ravindra Jadeja baowl','','https://stats.espncricinfo.com/ci/engine/player/234675.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Kedar Jadhav bat','https://stats.espncricinfo.com/ci/engine/player/290716.html?class=2;template=results;type=batting;view=innings',"",0,df_overall_bat,df_personal_bat],
            ['Kedar Jadhav bowl','','https://stats.espncricinfo.com/ci/engine/player/290716.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ['Hardik Pandya bat','https://stats.espncricinfo.com/ci/engine/player/625371.html?class=2;template=results;type=batting;view=innings',"",0,df_overall_bat,df_personal_bat],
            ['Hardik Pandya bowl','','https://stats.espncricinfo.com/ci/engine/player/625371.html?class=2;template=results;type=bowling;view=innings',1,df_overall_ball,df_personal_ball],
            ]


for a in all_players:
    Name=a[0]
    url1=a[1]
    url2=a[2]
    x=a[3]
    df1=a[4]
    df2=a[5]
    print(Name," Done.")
    returned_df,x= extract_data(Name,url1,url2, df1, df2,x)
    if (x==0):
        concat_elements=[df_overall_bat,returned_df]
        df_overall_bat=pd.concat(concat_elements, axis=0)
    if (x==1):
        concat_elements=[df_overall_ball,returned_df]
        df_overall_ball=pd.concat(concat_elements, axis=0)


Virat Kohli  Done.
Rohit Sharma  Done.
Ajinkya Rahane  Done.
Mayank Agarwal  Done.
Prithvi Shaw  Done.
Shikhar Dhawan  Done.
KL Rahul  Done.
Shreyas Iyer  Done.
Manish Pandey  Done.
Cheteshwar Pujara  Done.
MS Dhoni  Done.
Dinesh Karthik  Done.
Rishabh Pant  Done.
Deepak Chahar  Done.
Bhuvaneshwar Kumar  Done.
Shradul Thakur  Done.
Navdeep Saini  Done.
Mohammed Shami  Done.
Ishant Sharma  Done.
Umesh Yadav  Done.
Yuzvendra Chahal  Done.
Kuldeep Yadav  Done.
Jasprit Bumrah  Done.
R. Ashwin bat  Done.
R. Ashwin bowl  Done.
Ravindra Jadeja bat  Done.
Ravindra Jadeja baowl  Done.
Kedar Jadhav bat  Done.
Kedar Jadhav bowl  Done.
Hardik Pandya bat  Done.
Hardik Pandya bowl  Done.


In [14]:
df_overall_bat.head(3)

,Name,playing_span,matches_played,innings_batted,not_outs,runs_scored,highest_innings_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties,ducks,fours,sixes
0,Virat Kohli,2008-2020,248,239,39,11867,183,59.33,12726,93.25,43,58,13,1116,121
0,Rohit Sharma,2007-2020,224,217,32,9115,264,49.27,10250,88.92,29,43,13,817,244
0,Ajinkya Rahane,2011-2018,90,87,3,2962,111,35.26,3767,78.63,3,24,6,293,33


In [15]:
df_overall_ball.tail()

,Name,Span,Mat,Inns,Overs,Mdns,Runs,Wkts,best_bowling_in_innings,Ave,Econ,SR,4s,5s
0,Jasprit Bumrah,2016-2020,64,64,557.4,38,2541,104,5/27,24.43,4.55,32.1,5,1
0,R. Ashwin bowl,2010-2017,111,109,1003.3,35,4937,150,4/25,32.91,4.91,40.1,1,0
0,Ravindra Jadeja baowl,2009-2020,165,161,1396.1,50,6839,187,5/36,36.57,4.89,44.7,7,1
0,Kedar Jadhav bowl,2014-2020,73,42,197.5,1,1020,27,3/23,37.77,5.15,43.9,0,0
0,Hardik Pandya bowl,2016-2019,54,53,394.4,5,2195,54,3/31,40.64,5.56,43.8,0,0


# Data Cleaning & Adding Traditional Attributes

First we analyse 1 CSV of a Batsmen and 1 CSV of a bowler so that we can create a general function we can apply to each one of them. 

As the prime variable that we are predicting is the number of Runs in case of the batsmen and the number of wickets in the case of the Bowlers we have to Remove the cases in which they did not Bowl or did not get to bat.

For Batsmen-

No. of Innings: The number of innings in which the batsman has batted till the day of the match.
This attribute signifies the experience of the batsman. The more innings the batsman has played,
the more experienced the player is.

Strike Rate (SR): Strike rate is the average number of runs scored per 100 balls faced. In limited
overs cricket, it is important to score runs at a fast pace. More runs scored at a slow pace is rather
harmful to the team as they have a limited number of overs. This attribute indicates how quickly
the batsman can score runs

Centuries: Number of innings in which the batsman scored more than 100 runs. This attribute
indicates the capability of the player to play longer innings and score more runs.
Fifties: Number of innings in which the batsman scored more than 50 (and less than 100)
runs.This attribute indicates the capability of the player to play longer innings and score more runs.

Zeros: Number of innings in which the batsman was dismissed without scoring a single run. This
attribute shows how many times the batsman failed to score runs, hence this being a negative
factor, was impacts the batsman’s prediction negatively.

In [139]:
# Cleaning Algorithm.
def clean_batsmen(df1, x):
    df1.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
    for a in df1.index:
        if(df1['Dismissal'][a]=='run out' or df1['Dismissal'][a]=='caught' or df1['Dismissal'][a]=='lbw' or df1['Dismissal'][a]=='bowled' or df1['Dismissal'][a]=='stumped' or 
           df1['Dismissal'][a]=='hit wicket'):
            df1['Dismissal'][a]=1
        if(df1['Dismissal'][a]=='not out' or df1['Dismissal'][a]=='retired notout'):
            df1['Dismissal'][a]=0
        if(df1['Runs'][a]=='DNB' or df1['Runs'][a]=='TDNB'):
            df1.drop(index=a, inplace=True)
    opp=['v Bangladesh', 'v Pakistan', 'v Sri Lanka', 'v West Indies',
           'v New Zealand', 'v Zimbabwe', 'v South Africa', 'v England',
           'v Australia', 'v Bermuda', 'v Africa XI', 'v Scotland',
           'v Hong Kong', 'v Ireland', 'v Netherlands', 'v U.A.E.',
           'v Afghanistan']
    values=[0.3,0.6,0.9,1.2,1.5,1.8,2.1,2.4,2.7,3.0,3.3,3.6,3.9,4.2,4.5,4.8,5.1]
    a=0
    for val in opp:
        for i in df1.index:
                if(df1['Opposition'][i]==val):
                    df1['Opposition'][i]=values[a]
        a=a+1
    for val in df1.columns:
        for i in df1.index:
            if(df1[val][i]=='-'):
                df1[val][i]=0
    for a in df1.index:
        x= re.findall(r'\d+',df1['Runs'][a])
        df1['Runs'][a]=int(x[0])
    for a in ['Runs','Mins', 'BF', '4s', '6s', 'Pos', 'Dismissal', 'Inns']:
        df1[a]= df1[a].astype(int)
    df1['SR']= df1['SR'].astype(float)
    df1['Opposition']= df1['Opposition'].astype(float)
    #Cleaning Done.
    
    #Calculating Derived Attributes
    x= df1.shape
    count=x[0]
    num_diss=0
    cen=0
    fifty=0
    zero=0
    for i in df1.index:
        if(df1['Dismissal'][i]==1):
            num_diss= num_diss+1
        if(df1['Runs'][i]==0):
            zero=zero+1
        if(df1['Runs'][i]>50 and df1['Runs'][i]<100):
            fifty=fifty+1
        if(df1['Runs'][i]>100):
            cen=cen+1
    avg= df1['Runs'].sum(axis=0)/num_diss
    sr= df1["SR"].sum()/count
    Consis= 0.4262*avg + 0.2566*count + 0.1510*sr + 0.0787*cen +0.0556*fifty-0.0328*zero
    if(Consis>0 and Consis<49):
        df1['conistency']=1
    if(Consis>50 and Consis<74):
        df1['conistency']=2
    if(Consis>75 and Consis<99):
        df1['conistency']=3
    if(Consis>100 and Consis<124):
        df1['conistency']=4
    if(Consis>125):
        df1['conistency']=5
    return df1


#### Taking the top 5 Batsmen

In [151]:
df_virat= pd.DataFrame()
df_Rohit=pd.DataFrame()
df_Dhoni=pd.DataFrame()
df_Rahane=pd.DataFrame()
df_Dhawan=pd.DataFrame()
df_Rahul= pd.DataFrame()
Batsmen=['MS Dhoni','Virat Kohli', "Rohit Sharma","Ajinkya Rahane",'Shikhar Dhawan','KL Rahul']
different_df=[df_Dhoni,df_virat,df_Rohit,df_Rahane, df_Dhawan, df_Rahul]
for i in range(0,6):
    df1= pd.read_csv('c:ICT/'+Batsmen[i]+'.csv')
    x=i
    df1.insert(0,'Player ID', x)
    temp = clean_batsmen(df1, x)
    different_df[i]=temp
    print('Cleaned Dataframe for '+Batsmen[i])

Batsmen_df= pd.concat(different_df, axis=0)
print('Done')

Cleaned Dataframe for MS Dhoni
Cleaned Dataframe for Virat Kohli
Cleaned Dataframe for Rohit Sharma
Cleaned Dataframe for Ajinkya Rahane
Cleaned Dataframe for Shikhar Dhawan
Cleaned Dataframe for KL Rahul
Done


In [152]:
Batsmen_df= shuffle(Batsmen_df)

In [153]:
Batsmen_df.head()

,Player ID,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,conistency
201,1,113,166,106,9,1,106.60,3,1,1,1.5,Kanpur,4
138,2,150,215,133,13,6,112.78,1,1,2,2.1,Kanpur,3
67,3,33,59,34,4,2,97.05,2,1,2,1.5,Dharamsala,1
186,0,33,41,36,2,0,91.66,7,1,1,2.4,Chester-le-Street,4
4,2,29,68,43,5,0,67.44,4,1,1,2.7,Brisbane,3


## Normalizing Values

According to the Research Paper-

Both the problems are targeted as classification problems where number of runs and number of
wickets are classified in different ranges. We used naïve bayes, random forest, multiclass SVM and decision
tree classifiers to generate the prediction models for both the problems. Random Forest classifier was
found to be the most accurate for both the problems.

In [154]:
Batsmen_df.head()

,Player ID,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,conistency
201,1,113,166,106,9,1,106.60,3,1,1,1.5,Kanpur,4
138,2,150,215,133,13,6,112.78,1,1,2,2.1,Kanpur,3
67,3,33,59,34,4,2,97.05,2,1,2,1.5,Dharamsala,1
186,0,33,41,36,2,0,91.66,7,1,1,2.4,Chester-le-Street,4
4,2,29,68,43,5,0,67.44,4,1,1,2.7,Brisbane,3


In [155]:
Batsmen_df.loc[ Batsmen_df['Runs'] <= 50, 'Runs'] = 1,
Batsmen_df.loc[(Batsmen_df['Runs'] > 50) & (Batsmen_df['Runs'] <= 75), 'Runs'] = 2,
Batsmen_df.loc[(Batsmen_df['Runs'] > 75) & (Batsmen_df['Runs'] <= 100), 'Runs'] = 3,
Batsmen_df.loc[(Batsmen_df['Runs'] > 100) & (Batsmen_df['Runs'] <= 150), 'Runs'] = 4,
Batsmen_df.loc[(Batsmen_df['Runs'] > 150) & (Batsmen_df['Runs'] <= 200), 'Runs'] = 5,
Batsmen_df.loc[ Batsmen_df['Runs'] > 200, 'Runs'] = 6

In [156]:
Batsmen_df.loc[ Batsmen_df['Mins'] <= 50, 'Mins'] = 1,
Batsmen_df.loc[(Batsmen_df['Mins'] > 50) & (Batsmen_df['Mins'] <= 75), 'Mins'] = 2,
Batsmen_df.loc[(Batsmen_df['Mins'] > 75) & (Batsmen_df['Mins'] <= 100), 'Mins'] = 3,
Batsmen_df.loc[(Batsmen_df['Mins'] > 100) & (Batsmen_df['Mins'] <= 150), 'Mins'] = 4,
Batsmen_df.loc[(Batsmen_df['Mins'] > 150) & (Batsmen_df['Mins'] <= 200), 'Mins'] = 5,
Batsmen_df.loc[ Batsmen_df['Mins'] > 200, 'Mins'] = 6

In [157]:
Batsmen_df.loc[ Batsmen_df['BF'] <= 50, 'BF'] = 1,
Batsmen_df.loc[(Batsmen_df['BF'] > 50) & (Batsmen_df['BF'] <= 75), 'BF'] = 2,
Batsmen_df.loc[(Batsmen_df['BF'] > 75) & (Batsmen_df['BF'] <= 100), 'BF'] = 3,
Batsmen_df.loc[(Batsmen_df['BF'] > 100) & (Batsmen_df['BF'] <= 150), 'BF'] = 4,
Batsmen_df.loc[(Batsmen_df['BF'] > 150), 'BF'] = 5

In [158]:
Batsmen_df.loc[ Batsmen_df['4s'] <= 5, '4s'] = 1,
Batsmen_df.loc[(Batsmen_df['4s'] > 5) & (Batsmen_df['4s'] <= 10), '4s'] = 2,
Batsmen_df.loc[(Batsmen_df['4s'] > 15) & (Batsmen_df['4s'] <= 20), '4s'] = 3,
Batsmen_df.loc[(Batsmen_df['4s'] > 20) & (Batsmen_df['BF'] <= 25), '4s'] = 4,
Batsmen_df.loc[(Batsmen_df['4s'] > 25), '4s'] = 5

In [159]:
Batsmen_df.loc[ Batsmen_df['6s'] <= 2, '6s'] = 1,
Batsmen_df.loc[(Batsmen_df['6s'] > 2) & (Batsmen_df['6s'] <= 4), '6s'] = 2,
Batsmen_df.loc[(Batsmen_df['6s'] > 4) & (Batsmen_df['6s'] <= 6), '6s'] = 3,
Batsmen_df.loc[(Batsmen_df['6s'] > 6) & (Batsmen_df['6s'] <= 8), '6s'] = 4,
Batsmen_df.loc[(Batsmen_df['6s'] > 10), '6s'] = 5

In [160]:
Batsmen_df.loc[ Batsmen_df['SR'] <= 50, 'SR'] = 1,
Batsmen_df.loc[(Batsmen_df['SR'] > 50) & (Batsmen_df['SR'] <= 75), 'SR'] = 2,
Batsmen_df.loc[(Batsmen_df['SR'] > 75) & (Batsmen_df['SR'] <= 100), 'SR'] = 3,
Batsmen_df.loc[(Batsmen_df['SR'] > 100) & (Batsmen_df['SR'] <= 150), 'SR'] = 4,
Batsmen_df.loc[(Batsmen_df['SR'] > 150) & (Batsmen_df['SR'] <= 200), 'SR'] = 5,
Batsmen_df.loc[ Batsmen_df['SR'] > 200, 'SR'] = 6

In [161]:
Batsmen_df.head()

,Player ID,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,conistency
201,1,4,5,4,2,1,4.0,3,1,1,1.5,Kanpur,4
138,2,4,6,4,13,3,4.0,1,1,2,2.1,Kanpur,3
67,3,1,2,1,1,1,3.0,2,1,2,1.5,Dharamsala,1
186,0,1,1,1,1,1,3.0,7,1,1,2.4,Chester-le-Street,4
4,2,1,2,1,1,1,2.0,4,1,1,2.7,Brisbane,3


## Training And Testing Split

A 80:20 Training to testing Ratio is considerd

In [162]:
Batsmen_df.columns

Index(['Player ID', 'Runs', 'Mins', 'BF', '4s', '6s', 'SR', 'Pos', 'Dismissal',
       'Inns', 'Opposition', 'Ground', 'conistency'],
      dtype='object')

In [164]:
X=Batsmen_df[['Mins', 'BF', '4s', '6s', 'SR', 'Pos', 'Dismissal','Inns', 'Opposition','conistency']]
y=Batsmen_df['Runs'].values
x= preprocessing.StandardScaler().fit(X).transform(X)
x[0:5]

array([[ 2.08625383,  2.19174493,  0.03662798, -0.22730128,  1.19615123,
        -0.25720944,  0.45148303, -1.0809563 , -0.21535854,  0.79171284],
       [ 2.79556601,  2.19174493,  4.08182236,  3.3665646 ,  1.19615123,
        -1.35143944,  0.45148303,  0.92510678,  0.45095879, -0.15796421],
       [-0.04168269, -0.69732979, -0.33111696, -0.22730128,  0.31955573,
        -0.80432444,  0.45148303,  0.92510678, -0.21535854, -2.05731832],
       [-0.75099486, -0.69732979, -0.33111696, -0.22730128,  0.31955573,
         1.93125055,  0.45148303, -1.0809563 ,  0.78411746,  0.79171284],
       [-0.04168269, -0.69732979, -0.33111696, -0.22730128, -0.55703977,
         0.28990556,  0.45148303, -1.0809563 ,  1.11727613, -0.15796421]])

In [165]:
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.2, random_state=4)
print("Trainig Data=", x_train.shape,y_train.shape)
print("Testing data=", x_test.shape,y_test.shape)

Trainig Data= (803, 10) (803,)
Testing data= (201, 10) (201,)


# Testing Algorithms

## Random Forest

In [169]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [167]:
rfc = RandomForestClassifier(n_jobs=-1)
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
                                  n_jobs=-1, n_iter=nbr_iter, cv=5)
    #CV = Cross-Validation ( here using Stratified KFold CV)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params, ht_score

rf_parameters, rf_ht_score = hypertuning_rscv(rfc, param_grid,100, x_train, y_train)
print("Best parameters=",rf_parameters)
print("Best Accuracy", rf_ht_score)

Best parameters= {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': None, 'bootstrap': False}
Best Accuracy 0.8878959627329192


In [170]:
rfc = RandomForestClassifier(n_estimators=400, min_samples_split=10, min_samples_leaf=2, max_features='auto', 
                             max_depth=None, bootstrap=False,n_jobs=-1)
rfc.fit(x_train,y_train)
y_pred= rfc.predict(x_test)
accuracy_rf= accuracy_score(y_pred,y_test)

In [171]:
accuracy_rf

0.8805970149253731

# Decision Trees

In [172]:
from sklearn.tree import DecisionTreeClassifier

In [173]:
dt= DecisionTreeClassifier()

criterion=["gini", "entropy"]
splitter=["best", "random"]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2,3, 4, 5, 6, 7,8, 9, 10]
min_samples_leaf = [1, 2,3, 4]
max_features=["auto", "sqrt", "log2", None]

tree_grid= dict(criterion=criterion, splitter=splitter, max_depth=max_depth, min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf ,max_features=max_features)

rnd_tree=RandomizedSearchCV(dt,tree_grid, n_iter=100, cv=5, random_state=1)
model= rnd_tree.fit(x_train,y_train)

print("Best Parameters=",model.best_params_)
print("Accuracy Score=", model.best_score_)

Best Parameters= {'splitter': 'random', 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 90, 'criterion': 'gini'}
Accuracy Score= 0.8754192546583852


In [174]:
dt= DecisionTreeClassifier( splitter='random', min_samples_split= 10, min_samples_leaf= 1, max_features= None, 
                             max_depth= 90, criterion='gini')
dt.fit(x_train,y_train)
y_hat= dt.predict(x_test)

accuracy_dt= accuracy_score(y_hat,y_test)

In [175]:
accuracy_dt

0.900497512437811